# Loading Modules

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from collections import Counter

# Class Node

In [ ]:
class Node:
    def __init__(self, Level, Feature, Inequality, Gini, samples, values, Class):
        self.Level = Level
        self.Feature = Feature
        self.Inequality = Inequality
        self.Gini = Gini
        self.samples = samples
        self.values = values
        self.Class = Class
        self.left = None
        self.right = None
    
    def __repr__(self):
        return (str(self.Feature)+" <= "+str(self.Inequality))
    
    def print_info(self):
        print("--------------------------------------------------------")        
        print(f"Level {self.Level}")
        if not (self.Feature == "" or self.Inequality == -1):
            print(f"{self.Feature} <= {self.Inequality}")
        print(f"gini = {self.Gini}")
        print(f"samples = {self.samples}")
        print(f"value = {self.values}")
        print(f"class = {self.Class}")
        print("--------------------------------------------------------")
        print()

# Loading Iris Dataset

In [ ]:
iris = datasets.load_iris()

iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [ ]:
df = pd.DataFrame(iris.data)
Y = pd.DataFrame(iris.target)

print(iris.feature_names)
print()

df.columns = iris.feature_names              #adding feature names.

print(df.describe())
print(df.values.shape)

df["flower_type"] = Y                        # adding flower_type(output) column to df
                                             # so that subsets of df & y can be obtained together.
print(df.values.shape)
print(df.describe())

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         150.000000        150.000000         150.000000   
mean            5.843333          3.057333           3.758000   
std             0.828066          0.435866           1.765298   
min             4.300000          2.000000           1.000000   
25%             5.100000          2.800000           1.600000   
50%             5.800000          3.000000           4.350000   
75%             6.400000          3.300000           5.100000   
max             7.900000          4.400000           6.900000   

       petal width (cm)  
count        150.000000  
mean           1.199333  
std            0.762238  
min            0.100000  
25%            0.300000  
50%            1.300000  
75%            1.800000  
max            2.500000  
(150, 4)
(150, 5)
       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count        

# Gini Function

In [ ]:
def gini(y_split, D):
    Count = Counter(y_split)                # using Counter() to get the frequency of each flower_type index
    Set = set(y_split)
    giniD = 1                               # calculating Gini Index = 1 - [(sum of)Pi^2]
    for k in Set:
        giniD -= (Count[k]/D)**2
    return round(giniD, 3)

# Selecting the best feature and range to split across
(using gini index in this case)

In [ ]:
# The steps followed:-
#              *Find best value to split upon in each feature
#              *using gini index to find best value
#              *update best feature and value with each traversal

def best_feature(df, y):
    opt_gini = -1                  # to store best value of gini
    D = y.shape[0]
    giniD = gini(y, D)             # calling gini() fn. to calculate Gini index

    for i in iris.feature_names:   # iterating over each feature:- 'sepal length (cm)', 'sepal width (cm)',
                                   # 'petal length (cm)', 'petal width (cm)'
        x = df[i]
                                           
        # as the features are continous taking the mid point between each point and storing it in 'mpts'
        # ith mid point = sum(x[i:i+2]/2) and rounding it upto 3 decimal places
        mpts = set([round(sum((x[i:i+2])/2), 3) for i in range(0, len(x)-1)])
        
        for j in mpts:                         #  Iterating over each mid point and dividng the data points in 2 parts:-
            Split1 = df[x <= j]                # 1)lesser than the mid point
            Split2 = df[x > j]                 # 2)Greater than the mid point
            # converting into numpy array
            split1 = np.array(Split1)
            split2 = np.array(Split2)
            
            # slicing data
            y_split1 = split1[:, split1.shape[1]-1]
            y_split2 = split2[:, split2.shape[1]-1]
        
            D1 = y_split1.shape[0]
            D2 = y_split2.shape[0]
            
            # calculating gini index for each new "decision box" 
            giniD1 = gini(y_split1, D1)
            giniD2 = gini(y_split2, D2)
            
            gini_split = (D1/D)*giniD1 + (D2/D)*giniD2
            del_gini = giniD - gini_split
            
            # updating the value if found better            
            if(del_gini > opt_gini):
                Ans_Split1 = Split1
                Ans_Split2 = Split2
                opt_gini = del_gini
                best_feat = i
                best_range = j
                                      # returning Gini index of the main decision box, the feature across which we split,
                                      # the range across which we split, and DataFrame of the two splits
    return [giniD, best_feat, best_range, Ans_Split1, Ans_Split2]            

# Build Tree Function

#### helper() fn. will calculate "value" and "class" for the build_tree() fn. both of which will be printed in the final ouput

In [ ]:
def helper(y):
    Count = Counter(y)                            # creating a counter for the current 'y'
    di = {}                                       # creating a dictionary to store frequency of respective flower_type
    for i in Index:                               # using 'Index' from the cell above
        di[iris.target_names[int(i)]] =  Count[i]
    value = str(di)[1:-1]                            # Removing the curly braces('{' & '}') and storing it as string

    mc = Count.most_common(1)                     # getting the feature with highest frequency
    Class = iris.target_names[int(mc[0][0])]      # as y contains index values, accessing the 'flower_type' using the index
    return value, Class

In [ ]:
def build_tree(df, y, unused_features, Index, level):
# Base Cases:-
    # 1. y contains only one distinct value
    if(len(set(y)) == 1):
        value, Class = helper(y)
        
        # creating object Node with the obtained data
        root = Node(level, "", -1, 0.0, y.shape[0], value, Class)
        return root

#     # 2. unused is empty       *will never hit*(explaination in red)
#     elif(len(unused_features) == 0):
#         value, Class = helper(y)
        
#         # creating object Node with the obtained data
#         root = Node(level, "", -1, gini(y, y.shape[0]), y.shape[0], value, Class)
#         return root
        
    # calling 'best_feature()' to calculate the best feature and range
    ans = best_feature(df, y)
    curr_gini, best_feat, best_range, Split1, Split2 = ans[0], ans[1], ans[2], ans[3], ans[4]
    
    value, Class = helper(y)
    
    # creating object Node with the obtained data
    root = Node(level, best_feat, best_range, curr_gini, y.shape[0], value, Class)
    
    # remove best feature from unused features
    ''' we don't need to remove any feature because after each split over let's say r (x <= r & x > r), so each split
    will exclude r as a mid point candidate to split upon because it will never be the mid point in the new split, so 
    the 2nd base case will never hit. '''
    
    # slicing 'y' from split1 and split2
    split1 = np.array(Split1)
    split2 = np.array(Split2)
    y_Split1 = split1[:, split1.shape[1]-1]
    y_Split2 = split2[:, split2.shape[1]-1]

    # recursive calls
    root.left = build_tree(Split1, y_Split1, unused_features, Index, level+1)
    root.right = build_tree(Split2, y_Split2, unused_features, Index, level+1)
    
    return root

#### 'y' will contain numpy array of target, 'Index' will have a set with values {0, 1, 2}, 'unused_features' will have all the features in iris dataset ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'].

In [ ]:
y = (iris.target)
Index = set(y)               # will be used to iterate over in the build tree function
unused_features = set(iris.feature_names)

#### calling build_tree() and storing return value in Root which will be the root node of our tree.

In [ ]:
Root = build_tree(df, y, unused_features, Index, 0)       # passing x as DataFrame df, y as numpy array, unused_features as
                                                          # set, Index is also set, and Level as int starting from 0.

#### function to print the data of tree with it's  traversal.

In [ ]:
def traversal(root):
    if root == None:
        return
    root.print_info()
    traversal(root.left)
    traversal(root.right)

In [ ]:
traversal(Root)              # calling the traversal() fn.

--------------------------------------------------------
Level 0
petal width (cm) <= 0.8
gini = 0.667
samples = 150
value = 'setosa': 50, 'versicolor': 50, 'virginica': 50
class = setosa
--------------------------------------------------------

--------------------------------------------------------
Level 1
gini = 0.0
samples = 50
value = 'setosa': 50, 'versicolor': 0, 'virginica': 0
class = setosa
--------------------------------------------------------

--------------------------------------------------------
Level 1
petal width (cm) <= 1.75
gini = 0.5
samples = 100
value = 'setosa': 0, 'versicolor': 50, 'virginica': 50
class = versicolor
--------------------------------------------------------

--------------------------------------------------------
Level 2
petal length (cm) <= 4.9
gini = 0.168
samples = 54
value = 'setosa': 0, 'versicolor': 49, 'virginica': 5
class = versicolor
--------------------------------------------------------

---------------------------------------------